In [13]:
#%pip install xgboost
#%pip install wordcloud
#%pip install gensim

In [14]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score
## for data
import pandas as pd

import re
import nltk## for language detection

In [15]:
train = pd.read_csv('c:/data/project/train.csv')
test = pd.read_csv('c:/data/project/test.csv')

train.head()

,ID,first_party,second_party,facts,first_party_winner
0,TRAIN_0000,Phil A. St. Amant,Herman A. Thompson,"On June 27, 1962, Phil St. Amant, a candidate ...",1
1,TRAIN_0001,Stephen Duncan,Lawrence Owens,Ramon Nelson was riding his bike when he suffe...,0
2,TRAIN_0002,Billy Joe Magwood,"Tony Patterson, Warden, et al.",An Alabama state court convicted Billy Joe Mag...,1
3,TRAIN_0003,Linkletter,Walker,Victor Linkletter was convicted in state court...,0
4,TRAIN_0004,William Earl Fikes,Alabama,"On April 24, 1953 in Selma, Alabama, an intrud...",1


In [16]:
df = train[['facts', 'first_party_winner']]
df_target = df[['first_party_winner']]
df_nlp = df[['facts']]
df_nlp1 = pd.DataFrame(df_nlp, columns=['facts'])
df_nlp1['facts'] = df_nlp1['facts'].str.replace(r'<[^<>]*>', '', regex=True)

In [17]:
corpus = df_nlp1['facts']
# print(corpus.str.cat(sep=" ")) # 인덱스의 요소들 서로 잇기
lst_tokens = nltk.tokenize.word_tokenize(corpus.str.cat(sep=" "))
ps = nltk.stem.porter.PorterStemmer()
lem = nltk.stem.wordnet.WordNetLemmatizer()
lst_stopwords = nltk.corpus.stopwords.words("english")

In [18]:
def utils_preprocess_text(text, flg_stemm=False, flg_lemm=True, lst_stopwords=None):
    ## clean (convert to lowercase and remove punctuations and characters and then strip)
    import re
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
            
    ## Tokenize (convert from string to list)
    lst_text = text.split()    ## remove Stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in 
                    lst_stopwords]
                
    ## Stemming (remove -ing, -ly, ...)
    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]
                
    ## Lemmatisation (convert the word into root word)
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]
            
    ## back to string from list
    text = " ".join(lst_text)
    return text

In [19]:
df_nlp1["facts_clean"] = df_nlp1["facts"].apply(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=True, lst_stopwords=lst_stopwords))
df_nlp2 = pd.concat([df_nlp1,df_target['first_party_winner']],axis=1, join='inner')
df_nlp2

,facts,facts_clean,first_party_winner
0,"On June 27, 1962, Phil St. Amant, a candidate ...",june 27 1962 phil st amant candidate public of...,1
1,Ramon Nelson was riding his bike when he suffe...,ramon nelson riding bike suffered lethal blow ...,0
2,An Alabama state court convicted Billy Joe Mag...,alabama state court convicted billy joe magwoo...,1
3,Victor Linkletter was convicted in state court...,victor linkletter convicted state court eviden...,0
4,"On April 24, 1953 in Selma, Alabama, an intrud...",april 24 1953 selma alabama intruder broke apa...,1
...,...,...,...
2473,Congress amended the Clean Air Act through the...,congress amended clean air act energy policy a...,1
2474,"Alliance Bond Fund, Inc., an investment fund, ...",alliance bond fund inc investment fund purchas...,1
2475,"In 1992, the District Court sentenced Manuel D...",1992 district court sentenced manuel peguero 2...,0
2476,"On March 8, 1996, Enrico St. Cyr, a lawful per...",march 8 1996 enrico st cyr lawful permanent re...,0


In [20]:
vectorize=CountVectorizer()

In [21]:
count_matrix = vectorize.fit_transform(df_nlp2['facts_clean'])
count_array = count_matrix.toarray()
data_final = pd.DataFrame(data=count_array,columns = vectorize.get_feature_names_out())
data_final = pd.concat([data_final,df_nlp2["first_party_winner"]],axis=1,join='inner')
data_final

,011119,0125,02,036539,04,041352,041581,0479,05,0511287,...,zone,zoned,zoneofinterests,zoning,zubik,zuni,zurich,zurko,zurkos,first_party_winner
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2473,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2474,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2475,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2476,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
X_train, X_test, y_train, y_test = train_test_split(data_final.drop(columns=['first_party_winner']), data_final['first_party_winner'], test_size=0.3,random_state=0)

In [51]:
# X_train = pd.DataFrame(data_final.drop(columns=['first_party_winner']))
# y_train = pd.DataFrame(data_final['first_party_winner'])

In [52]:
# 1분정도 걸림

from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=200, random_state=0)
lda_data = lda.fit_transform(X_train)

In [53]:
lda_data_train = pd.DataFrame(data=lda_data)

In [64]:
model = XGBClassifier(                      booster = 'gbtree',
                      scale_pos_weight=5,
                      learning_rate=0.01,  
                      colsample_bytree = 0.7,
                      subsample = 0.5,
                      max_delta_step = 3,
                      reg_lambda = 2,
                     objective='binary:logistic',
                      n_estimators=818, 
                      max_depth=8,
)
model.fit(lda_data_train, y_train) # x,y

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [65]:
# from sklearn.metrics import f1_score
# f1_score()

In [66]:
lda_data_test = pd.DataFrame(data=lda.transform(X_test))
y_pred = model.predict(lda_data_test)
predictions = [round(value) for value in y_pred]

In [67]:
# evaluate predictions
from sklearn.metrics import f1_score
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
f1_score(y_test, y_pred)

Accuracy: 17.74%


0.0

# 테스트 적용

In [68]:
dfTest = pd.DataFrame(test['facts'])
dfTest['facts'] = dfTest['facts'].str.replace(r'<[^<>]*>', '', regex=True)
dfTest["facts_clean"] = dfTest["facts"].apply(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=True, lst_stopwords=lst_stopwords))
dfTest['first_party_winner'] = np.zeros(len(dfTest)).astype(int)
dfTest

,facts,facts_clean,first_party_winner
0,The 1984 Bail Reform Act allowed the federal c...,1984 bail reform act allowed federal court det...,0
1,Lexecon Inc. was a defendant in a class action...,lexecon inc defendant class action lawsuit 28 ...,0
2,"In 2002 and 2003, Fox Television Stations broa...",2002 2003 fox television station broadcast bil...,0
3,During his trial for armed robbery of a federa...,trial armed robbery federally insured saving l...,0
4,"In 1993, a magistrate judge issued a warrant a...",1993 magistrate judge issued warrant authorizi...,0
...,...,...,...
1235,According to Executive Order No. 12807 signed ...,according executive order 12807 signed preside...,0
1236,Section 109(a) of the Clean Air Act (CAA) requ...,section 109a clean air act caa requires enviro...,0
1237,Linda Matteo and John Madigan created a plan f...,linda matteo john madigan created plan utilizi...,0
1238,"In 1972, the North Carolina Board of Agricultu...",1972 north carolina board agriculture adopted ...,0


In [69]:
count_matrix_test = vectorize.transform(dfTest['facts_clean'])
count_array_test = count_matrix_test.toarray()
data_final_test = pd.DataFrame(data=count_array_test,columns = vectorize.get_feature_names_out())
data_final_test = pd.concat([data_final_test,dfTest["first_party_winner"]],axis=1,join='inner')
data_final_test

,011119,0125,02,036539,04,041352,041581,0479,05,0511287,...,zone,zoned,zoneofinterests,zoning,zubik,zuni,zurich,zurko,zurkos,first_party_winner
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1235,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1236,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1237,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1238,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [70]:
X_test = data_final_test.drop(columns=['first_party_winner'])
y_test = data_final_test['first_party_winner']

In [71]:
lda_data_test2 = pd.DataFrame(data=lda.transform(X_test))

In [72]:
y_pred_model_test = model.predict(lda_data_test2)
predictionsTest = [round(value) for value in y_pred_model_test]
predcsv = pd.DataFrame(predictionsTest,columns=['first_party_winner'])

In [73]:
# submit = pd.read_csv('C:/data/project/sample_submission.csv')
# submit['first_party_winner'] = predcsv
# submit.to_csv('./sample_submission.csv', index=False)
# print('Done')